# Ego-Lane-fitting-Pointclouds
+ Detects ego lane lines from pointcloud data, focusing on the top view to fit a 3-degree polynomial for the left and right lane lines. 
+ The algorithm's output must be the polynomial coefficients for each lane line, formatted similarly to the provided sample output.

### Data Handling
+ `Pointcloud Data`: 
    * work with binary files containing pointcloud data. 
    * Each point:  x, y, z, intensity, lidar beam values. 
    * The intensity and lidar beam values are crucial for distinguishing lane points.
+ `Visualization`: The task provides a `data_visualize.py` script for visualizing pointcloud data, which can be a valuable tool for debugging and validating your algorithm's output.

### Approach and Methodology
Given a small dataset, traditional machine learning and geometric algorithms are more suitable for processing LiDAR pointcloud data for tasks such as ego lane detection. 
+ `Preprocessing`: 
    * Clean and filter the pointcloud data, focusing on points likely to represent lane markings based on intensity and possibly z values.
+ `Lane Point Classification`: 
    * Use intensity and other heuristics to distinguish between lane and non-lane points. 
        * reflective scatter intensity and frequency of the lane is generally larger and higher than non-lane points (white/yellow for lane)
        * threshold values obtain by exploiting clustering techniques to identify points belonging to lane markings accurately.
+ `Lane Line Fitting`: Once you have identified lane points, 
    * fit a 3-degree polynomial to these points for both the left and right lanes. 
    * use curve fitting techniques or optimization algorithms to find the best fit.

### Development
+ `Libraries`: 
    * Open3D or Matplotlib for visualization.
    * SciPy for fitting polynomials.
+ `Algorithm Implementation`: Develop your algorithm step by step, starting with 
    * data loading
    * data preprocessing
        * `DBSCAN (Density-Based Spatial Clustering of Applications with Noise)`: After filtering, clustering can help group the remaining points into distinct lane markings based on proximity. DBSCAN groups points closely packed together and marks points in low-density regions as outliers. It’s useful for segmenting objects from the background or separating different objects in a pointcloud.
        * `Z Fitting`: Filter scatters elevation higher than 90% of distribution
        * `Intensity-based Filtering`: Since lane markings often have higher reflectivity than the road surface, intensity values can be used to filter points likely to represent lane markings.
        * `Ground Segmentation (optional)`: Separate the ground plane from other objects using geometric methods like RANSAC or a simple plane fitting algorithm, which is essential for focusing on lane markings.
            * `RANSAC (Random Sample Consensus)`: Used for plane fitting and outlier removal. It's effective for identifying ground planes and other large flat surfaces by iteratively selecting a subset of points, fitting a model (e.g., a plane), and then measuring the model's validity against the entire dataset.
    * data detection
        * `Peaks Finding`: Find number of intensity peaks accross x-axis to estimate number of lanes, where the peak threshold is learned from each density based clusters
        * `KMeans Clustering`: Similar-slope lines bins segmentation using kmeans based on desired polynomial power degree and learned number of lanes
    * lane marking
        * `Slope-based Grid`: Create lane boundary based on orientation and intercept of each clustered line segmentation
    * lane polynomial fitting
        * `Polynomial Curve Fitting`: Once lane markings are identified, use methods like least squares to fit a polynomial curve to each detected lane marking. This step directly corresponds to the requirement of modeling lane lines with a 3-degree polynomial.

### Future Finen Tuning
#### Preprocessing
+ `Voxel Grid Downsampling`: Reduces the density of the pointcloud by averaging points within each voxel, which can help speed up computations without losing significant detail.
+ `Vertex normal estimation`: When the pointcloud will be used for surface reconstruction, rendering, or advanced analysis tasks
    1. `Surface Reconstruction and Meshing`
        For reconstructing surfaces from pointclouds, knowing the normals at each point is crucial. Normals indicate the orientation of the surface at each point, which helps algorithms like Poisson reconstruction or Greedy Triangulation to accurately generate meshes from the pointcloud data.
        Normals are used to determine the "inside" and "outside" of a surface, which is essential for creating solid objects from pointclouds.
    2. `Rendering and Visualization`
        When visualizing pointclouds, especially in 3D rendering software, normals are used to apply lighting and shading effects correctly. Normals help in determining how light reflects off surfaces, which enhances the visual perception of depth and material properties in the rendered scene.
    3. `Feature Extraction and Object Recognition`
        In some advanced analysis tasks, such as identifying specific objects or features within a pointcloud, normals can provide additional geometric information that complements the raw position data. For example, the orientation of surface elements can help differentiate between vertical walls and horizontal roads.
    4. `Improving Registration Accuracy`
        For pointcloud registration tasks (aligning two pointclouds), normals can improve the accuracy of algorithms like Iterative Closest Point (ICP). By considering both the position and orientation of points, these algorithms can achieve more precise alignments, especially in scenes with complex geometries.
    5. `Robustness to Noise and Downsampling`
        Estimating normals after downsampling can help mitigate the effects of noise in the original pointcloud. By working with a reduced set of points, the normal estimation process can focus on the underlying surface's main features, potentially leading to more accurate and stable normals.
#### Geometric and Heuristic Methods
+ `Hough Transform`: Often used in image processing for line detection, it can also be applied to pointcloud data for detecting lane lines or other linear features by transforming points into a parameter space and detecting collinear points.
#### Feature Extraction and Filtering Techniques
+ `PCA (Principal Component Analysis)`: Used for dimensionality reduction and feature extraction. It can help identify the main directions of variance in the data, useful for tasks like ground plane removal or object orientation estimation.
+ `Voxel Grid Filtering`: Reduces the resolution of pointcloud data by averaging points within voxel grids. It's a common preprocessing step to reduce computational load while preserving the overall structure of the scene.
#### Curve Fitting
+ `Polynomial Curve Fitting`: Employed for lane detection, as mentioned in your task, where polynomial functions are fitted to data points representing lane markings.
+ `Iterative Closest Point (ICP)`: An algorithm for aligning two pointclouds or a pointcloud and a model. It's used in tasks like SLAM (Simultaneous Localization and Mapping) for map building and localization.
#### Optimization
+ `Smoothing Filters`: Apply smoothing techniques to the polynomial coefficients to ensure lane lines are not overly sensitive to noise or outliers in the data. Techniques like moving averages or Savitzky-Golay filters can be effective.
####  Evlautation and Validation
+ `Manual Inspection`: Use visualization tools to manually inspect the fitted lane lines against the pointcloud data to ensure they accurately represent the ego lanes
+ `Cross-validation`: If some labeled data is available, cross-validation techniques can be used to evaluate the robustness of your algorithm and optimize parameters.
#### Machine Learning and Deep Learning Methods (Not Suitable)
+ `PointNet and Variants (PointNet++, DGCNN, etc.)`: Neural networks designed specifically for processing pointcloud data. They can classify, segment, and extract features from pointclouds directly, handling the data's unordered nature.
+ `Convolutional Neural Networks (CNNs)`: While traditionally used for image data, CNNs can be applied to pointclouds that have been projected onto a 2D plane (e.g., bird’s-eye view) or converted into voxel grids or range images.
+ `Graph Neural Networks (GNNs)`: Used for pointclouds modeled as graphs, where points are nodes and edges represent spatial or feature relationships. GNNs can capture complex patterns in the data for tasks like segmentation and object classification.

## Data Preprocessing
### Voxel downsampling
Voxel downsampling uses a regular voxel grid to create a uniformly downsampled point cloud from an input point cloud. It is often used as a pre-processing step for many point cloud processing tasks. The algorithm operates in two steps:
1. Points are bucketed into voxels.
2. Each occupied voxel generates exactly one point by averaging all points inside.
<br>
(source: open3D)
<br>

### Vertex normal estimation
Another basic operation for point cloud is point normal estimation. 
1. A normal at a point on a surface is a vector that is perpendicular (orthogonal) to the tangent plane at that point.
2. Normals indicate the orientation of the surface at each point, which helps algorithms like Poisson reconstruction or Greedy Triangulation to accurately generate meshes from the pointcloud data.
<br>
(source: open3D)
<br>

#### Note
In the context of ego lane detection, while normal estimation might not be directly necessary for identifying lane markings, understanding the procedure and its applications is valuable for broader pointcloud processing tasks and when considering the integration of pointcloud data into more complex scene analysis and reconstruction workflows.

### Z Filtering
Filter the scatters that is higher than ground and lanes by computing the mean and standard deviation of z

### Intensity-based Filtering Functions
Selecting points from the pointcloud based on their intensity values, isolating potential lane markings, which typically have higher reflectivity
1. apply `DBSCAN clustering` to group local point cloud clusters together to enable lanes intensity threshold learning
2. Correlate the DBSCAN labels with the intensity values stored in the attributes array.
3. For each cluster, based on the DBSCAN result, for each point cloud
    * Intensity Threshold Learning: Determine the intensity threshold based on the intensity of each cluster
4. Filter Clusters Based on Intensity 

#### DBSCAN Clustering Functions
* clustering
* tuning eps and min_samples hyperparas: approached through grid search, random search, or more sophisticated optimization techniques, but it often involves a trade-off between computational cost and the quality of the resulting clusters.

### Intensity Threshold

### Ground Plane Segmentation
simplify further analysis like lane marking detection by reducing the data's complexity. One common approach for ground segmentation is to use the Random Sample Consensus (RANSAC) algorithm
+  `distance_threshold`: 
    * defines the maximum distance a point can have to an estimated plane to be considered an inlier 
+ `ransac_n`:
    * defines the number of points that are randomly sampled to estimate a plane
+ `num_iterations`: 
    * defines how often a random plane is sampled and verified. 
The function then returns the plane as (a,b,c,d) such that for each point (x,y,z) on the plane we have ax +by + cz + d = 0. The function further returns a list of indices of the inlier points.

## 3D Point Cloud Visualizatoin

## Lane Detection
- `Line Clustering`
    * Polyline fitting requires lane clustering
    * K-means clustering based on euclidean distance is applied to cluster lanes and crosswalks.
    * the optimal k-mean cluster is determines based on the result of silhouette score nd visualized multiple cluster per k-mean value.
    * the clustering visulization will utilize matlibplot scatter plot
- `Lanes Detection`
    * detect numner of lanes based on number of peak intensity in the range of y
- `Line Fitting`
    * For each cluster that potentially represents a lane, perform a polynomial fit. You can use np.polyfit with a degree of 2 or 3 (for a quadratic or cubic fit), which is common for lane fitting.
- `Cost Function`
    * To refine the fits or to choose between multiple candidate fits, define a cost function that penalizes fits which deviate significantly from what a typical lane would look like. This can include factors such as lane width, parallelism with other lanes, and orientation relative to the driving direction.

### Line Clustering: K-Means Clustering
* `Scaler Transform`: standardize the data by transforming based on standardscaler


#### Optimize K Means:  Elbow or Silhouette?
* `Elbow Method`:
    * Principle: The elbow method looks at the percentage of variance explained as a function of the number of clusters. You plot the number of clusters against the within-cluster sum of squares (WCSS), looking for an "elbow" where the rate of decrease sharply changes. This point is considered to be indicative of the appropriate number of clusters.
    * Suitability for Elongated Features: The elbow method can sometimes struggle with identifying the correct number of clusters for data with elongated features because it relies on variance, which might not decrease significantly after a certain point if the clusters are elongated and dispersed.
    * Use Case: It's more suitable when the clusters have a roughly spherical shape rather than elongated features. However, it can still provide a good initial estimate or insight into the clustering tendency of the dataset.
* `Silhouette Method`:
    * Principle: The silhouette method measures how similar an object is to its own cluster compared to other clusters. The silhouette score ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.
    * Suitability for Elongated Features: The silhouette method can be more effective for evaluating the quality of clustering when dealing with elongated features. This is because it considers both cohesion (how close objects are to other objects in their cluster) and separation (how distinct a cluster is from other clusters), rather than just the variance within clusters.
    * Use Case: It's especially useful when the data contains complex structures or when the clusters are not spherical. The silhouette score provides a more nuanced view of cluster quality and separation, making it a better choice for assessing the appropriateness of the number of clusters in cases with elongated features.

## Lane Detection

## Lane Marking